In [2]:
import re
import pandas as pd

df = pd.read_excel("Your address")
df

,상품명,표준상품명
0,----------------,-
1,-----------------,-
2,----------------------,-
3,--------------------------------,-
4,도트 빗살삔,-
...,...,...
194862,ㅡ0.5 빼기,-
194863,ㅡ0.5단계 빼기,-
194864,ㅡ0.5단계 빼기+,-
194865,ㅣ버터구이한입쥐포,버터 쥐포


In [3]:
# 세트, 옵션 제거
remove_words = ["set", "세트", "셋트", "셑", "\+", "추가", "빼고", "빼기", "옵션"]

for word in remove_words:
    df.loc[df['표준상품명'].str.contains(word, case=False),'표준상품명'] = '-'

df

,상품명,표준상품명
0,----------------,-
1,-----------------,-
2,----------------------,-
3,--------------------------------,-
4,도트 빗살삔,-
...,...,...
194862,ㅡ0.5 빼기,-
194863,ㅡ0.5단계 빼기,-
194864,ㅡ0.5단계 빼기+,-
194865,ㅣ버터구이한입쥐포,버터 쥐포


In [ ]:
def preprocess_text(text):
    # 괄호와 괄호 안 단어 제거
    text = re.sub(r'\([^)]*\)|\{[^}]*\}|\[[^]]*\]', '', text)
    
    # 특수문자, 숫자 제거
    text = re.sub('[^가-힣a-zA-Z\s]', ' ', str(text))
    
    # 앞뒤 공백 제거
    text = text.strip()
    
    return text


# 특정 열에 함수 적용
df['표준상품명'] = df['표준상품명'].astype(str)
df['표준상품명'] = df['표준상품명'].apply(preprocess_text)

# 공백 1개로 통일
df['표준상품명'] = df['표준상품명'].apply(lambda x: re.sub(r'\s+', ' ', x))

pd.set_option('display.max_rows', 10000)
print(df['표준상품명'].tail(10000))

In [ ]:
import langid
from googletrans import Translator
from concurrent.futures import ThreadPoolExecutor

# 번역기 객체 생성
translator = Translator()

# 번역할 열 선택
target_col = '표준상품명'

# 번역 함수
def translate_text(text):
    try:
        if langid.classify(text)[0] == 'en':
            return translator.translate(str(text), src='auto', dest='ko').text
    except:
        return text
    
# 병렬처리 함수
def parallelize(data, func):
    with ThreadPoolExecutor() as executor:
        result = executor.map(func, data)
    return result

df[target_col] = list(parallelize(df[target_col], translate_text))

# 결과 출력
print(df['표준상품명'].tail(10000))

In [ ]:
def korean(text):
    # 영어 제거
    text = re.sub('[^가-힣\s]', '', str(text))
    
    # 앞뒤 공백 제거
    text = text.strip()
    
    return text


# 특정 열에 함수 적용
df['표준상품명'] = df['표준상품명'].astype(str)
df['표준상품명'] = df['표준상품명'].apply(korean)

# 공백 1개로 통일
df['표준상품명'] = df['표준상품명'].apply(lambda x: re.sub(r'\s+', ' ', x))

# 결과 출력
print(df['표준상품명'].tail(10000))

In [5]:
df.loc[df['표준상품명'] == 'nan', '표준상품명'] = '-'
df.loc[df['표준상품명'] == '', '표준상품명'] = '-'
df

,상품명,표준상품명
0,----------------,-
1,-----------------,-
2,----------------------,-
3,--------------------------------,-
4,도트 빗살삔,-
...,...,...
194862,ㅡ0.5 빼기,-
194863,ㅡ0.5단계 빼기,-
194864,ㅡ0.5단계 빼기+,-
194865,ㅣ버터구이한입쥐포,버터 쥐포


In [ ]:
from konlpy.tag import Okt

okt = Okt()

df['표준상품명'] = df['표준상품명'].apply(lambda x: ' '.join(okt.nouns(str(x))))
df

In [4]:
# 저장하기
df.to_excel("Your address", index=False)